In [1]:
import pandas as pd

In [317]:
restaurant_review = pd.read_csv('training-RestoInfo/RestoInfo.csv')
restaurant_review = restaurant_review.drop_duplicates()

In [318]:
import re

def replacenth(string, sub, wanted, n):
    where = [m.start() for m in re.finditer(sub, string)][n-1]
    before = string[:where]
    after = string[where:]
    after = after.replace(sub, wanted, 1)
    newString = before + after
    return newString

def fix_remover_eval(remover):
    bad_quotes = []
    for i,match in enumerate(re.finditer(r"\'", remover)):
        span_start, span_end = match.span()
        before = remover[span_start-2:span_start]
        after = remover[span_end:span_end+1]
        if before == '[(' or before == ' (' or before == ', ':
            continue
        if after == "," or after == ")":
            continue
        else:
            bad_quotes.append((i, match.group(0)))

    for i,q in bad_quotes:
        remover = replacenth(remover, "\'", "\\'", i+1)

    bad_quotes = []
    for i,match in enumerate(re.finditer(r'\"', remover)):
        span_start, span_end = match.span()
        before = remover[span_start-2:span_start]
        after = remover[span_end:span_end+1]
        if before == '[(' or before == ' (' or before == ', ':
            continue
        if after == "," or after == ")" or after == ', ':
            continue
        else:
            bad_quotes.append((i, match.group(0)))

    for i,q in bad_quotes:
        remover = replacenth(remover, '\"', '\\"', i+1)
    return eval(remover)

In [319]:
def get_eval(t):
    try:
        return fix_remover_eval(t)
    except:
        return t
    
restaurant_review['reviews_list_evaled'] = restaurant_review['reviews_list'].apply(get_eval)
restaurant_review['menu_item_evaled'] =restaurant_review['menu_item'].apply(eval)

In [320]:
restaurant_review_fixed = restaurant_review[restaurant_review['reviews_list_evaled'].apply(type) != str].rename(
    {"Unnamed: 0": "rest_id"}, axis=1)

In [322]:
restaurant_review_fixed['cuisines_list'] = restaurant_review_fixed[
    'cuisines'].fillna("").apply(lambda x: x.split(' ,'))
restaurant_review_fixed['dish_liked_list'] = restaurant_review_fixed[
    'dish_liked'].fillna("").apply(lambda x: x.split(' ,'))

In [348]:
recommendation_requirements = restaurant_review_fixed[['name',
    'reviews_list_evaled', 'menu_item_evaled', 'cuisines_list', 
    'listed_in(city)', 'approx_cost(for two people)', 'rate', 'dish_liked_list']].rename(
    {"listed_in(city)": "location", "approx_cost(for two people)": "cost"}, axis=1)

In [349]:
def float_converter(x):
    try:
        all_floats = re.findall(r'\d+', x)
        if all_floats:
            return float(all_floats[0])
        else:
            return -1
    except:
        return -1

recommendation_requirements['cost'] = recommendation_requirements['cost'].fillna("-1").apply(
    lambda x: int(x.replace(",", "")))

recommendation_requirements['rate'] = recommendation_requirements['rate'].apply(float_converter)

recommendation_requirements = recommendation_requirements.drop_duplicates(['name', 'location'])

In [648]:
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import tokenize

def get_words_ordered(s):
    if not s:
        return []
    return [w for w in tokenize(remove_stopwords(s.lower())) if w]


def get_words(s):
    if not s:
        return set()
    return set(get_words_ordered(s))

In [649]:
from gensim.models import Word2Vec

all_reviews = recommendation_requirements['reviews_list_evaled'].explode().dropna().apply(
    lambda x: x[1].strip('RATEDn  '))
all_reviews_tokenised = all_reviews.apply(get_words_ordered).tolist()
model = Word2Vec(all_reviews_tokenised, size=200, window=5, min_count=2, workers=8, iter=50)

KeyboardInterrupt: 

In [650]:
from sklearn.metrics.pairwise import cosine_similarity

In [676]:
def get_set_subset_standard(s1_set, s2):
    s2_set = get_words(s2.lower())
    return len(s1_set.intersection(s2_set))/len(s1_set)


def get_set_subset_wv(s1_set, s2):
    s2_list = get_words_ordered(s2.lower())
    s2_list = [w for w in s2_list if w in model.wv]
    if not s2_list:
        return 0
    s1_wv = model.wv[[w for w in s1_set if w in model.wv]]
    s2_wv = model.wv[s2_list]
    similarity_status = cosine_similarity(s1_wv, s2_wv)
    return similarity_status.max(axis=1).sum()/len(s1_set)
    

def get_set_subset(s1_set, s2):
    return get_set_subset_wv(s1_set, s2)

In [750]:
User_text = "good ambiance restaurants, serving fish"  
Location = "Koramangala"
budget = 500
cuisine_type = "North Indian"

In [751]:
User_text_refined = re.sub(r'(restaurants?|food|good|floor)', '', User_text.lower())
User_text_set = get_words(User_text_refined)
cuisine_type_set = get_words(cuisine_type)
location_lower = " ".join(get_words_ordered(Location))
User_text_set

{'ambiance', 'fish', 'serving'}

In [752]:
recommendation_requirements_custom = recommendation_requirements
if budget is not None :
    recommendation_requirements_custom = recommendation_requirements_custom[
        (recommendation_requirements_custom['cost'] <= budget) & (recommendation_requirements_custom['cost'] > 0)
    ]

In [753]:
location_satisfied = recommendation_requirements_custom['location'].str.lower().apply(
    lambda x: location_lower in x)
if location_satisfied.any():
    recommendation_requirements_custom = recommendation_requirements_custom[location_satisfied]

In [754]:
cusine_req_fullfilled = recommendation_requirements_custom[
    'cuisines_list'].apply(lambda x: cuisine_type in x)
if cusine_req_fullfilled.any():
    recommendation_requirements_custom = recommendation_requirements_custom[cusine_req_fullfilled]

In [755]:
menu_item_scored = recommendation_requirements_custom['menu_item_evaled'].apply(lambda x: max(
    [get_set_subset_standard(User_text_set, d) for d in x] + [0]))
dish_liked_scored = recommendation_requirements_custom['dish_liked_list'].apply(lambda x: max(
    [get_set_subset_standard(User_text_set, d) for d in x] + [0]))
review_scores = recommendation_requirements_custom['reviews_list_evaled'].apply(lambda x: 
    [float_converter(r)/5*get_set_subset(User_text_set, t) for r,t in set(x)])

In [756]:
import math
def get_review_scores(scores):
    num_review = len(scores)
    if not num_review:
        return 0
    multiplier = 1/(1+math.exp(-((num_review-7)/2.8)))
    return (sum(scores)/len(scores))*multiplier*(10/6)

In [757]:
dish_score = dish_liked_scored*0.4 + menu_item_scored*0.2
if dish_score.max() != 0:
    dish_score = dish_score/dish_score.max()
    recommendation_requirements_custom['final_score'] = dish_score*0.6 + \
        review_scores.apply(get_review_scores)*0.4
else:
    recommendation_requirements_custom['final_score'] = review_scores.apply(get_review_scores)

In [758]:
recommendation_requirements_custom.sort_values('final_score', ascending=False)[:3]

,name,reviews_list_evaled,menu_item_evaled,cuisines_list,location,cost,rate,dish_liked_list,final_score
1581,Paisa Vasool,"[(Rated 4.0, RATEDn It definitely goes by it'...",[],[North Indian],Koramangala 6th Block,400,3.0,"[Fish Thali, Aloo Paratha]",0.635304
875,Roti Wala,"[(Rated 4.0, RATEDn Well, I never had a chanc...",[],[North Indian],Koramangala 5th Block,300,4.0,"[Gulab Jamun, Buttermilk, Egg Bhurji, Sprout S...",0.075898
1554,Bowlful,"[(Rated 5.0, RATEDn Ordered food to my office...",[],[North Indian],Koramangala 4th Block,500,3.0,[],0.056398


In [744]:
dish_score.max()

1.0

In [745]:
review_scores.apply(get_review_scores).max()

0.9089325807953956

In [746]:
dish_liked_scored.max()

0.6666666666666666

In [747]:
get_set_subset_standard(User_text_set, 'Chicken Masala Biryani,')

0.6666666666666666

In [748]:
get_words('Chicken Masala Biryani,')

{'biryani', 'chicken', 'masala'}

In [749]:
User_text_set

{'authentic', 'biryani', 'chicken'}

In [702]:
5/3

1.6666666666666667